# Nettoyage et Analyse des Données CoinAfrique

Ce notebook permet de charger les données brutes extraites via Web Scraper, de les nettoyer (doublons, types de données, prix) et de les préparer pour le Dashboard Streamlit.

In [1]:
import pandas as pd


## 1. Chargement des données brutes

In [2]:
data1=pd.read_csv("donnée web scrapper/data_url1.csv")
data1.head()

,web_scraper_order,web_scraper_start_url,container_links,type_habits,price,adress,url_image
0,1771237466-1,https://sn.coinafrique.com/categorie/vetements...,https://sn.coinafrique.com/annonce/vetements-h...,Pantalon homme,Prix sur demande,"Dakar Plateau, Dakar, Sénégal","background-image: url(""https://images.coinafri..."
1,1771237468-2,https://sn.coinafrique.com/categorie/vetements...,https://sn.coinafrique.com/annonce/vetements-h...,Paquet Tshirt cotton 100℅,7 500 CFA,"Colobane, Dakar, Sénégal","background-image: url(""https://images.coinafri..."
2,1771237473-3,https://sn.coinafrique.com/categorie/vetements...,https://sn.coinafrique.com/annonce/vetements-h...,Ensemble homme,13 000 CFA,"Dakar Plateau, Dakar, Sénégal","background-image: url(""https://images.coinafri..."
3,1771237482-4,https://sn.coinafrique.com/categorie/vetements...,https://sn.coinafrique.com/annonce/vetements-h...,Jean homme,7 000 CFA,"Dakar Plateau, Dakar, Sénégal","background-image: url(""https://images.coinafri..."
4,1771237484-5,https://sn.coinafrique.com/categorie/vetements...,https://sn.coinafrique.com/annonce/vetements-h...,Ensemble homme,13 000 CFA,"Dakar Plateau, Dakar, Sénégal","background-image: url(""https://images.coinafri..."


## 2. Nettoyage des données

In [3]:
# j'enleve les colonnes inutiles
data1.drop(["web_scraper_order","web_scraper_start_url","container_links"], axis=1,inplace=True)


In [4]:
# je nettoie le prix
data1['price'] = data1['price'].str.replace(" CFA", "", regex=False).str.strip()

In [5]:
# je nettoie les url des imafes
data1['url_image'] = data1['url_image'].str.split('url("', regex=False).str[1]
data1['url_image'] = data1['url_image'].str.split('")', regex=False).str[0]
data1

,type_habits,price,adress,url_image
0,Pantalon homme,Prix sur demande,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/5305847_uploade...
1,Paquet Tshirt cotton 100℅,7 500,"Colobane, Dakar, Sénégal",https://images.coinafrique.com/5306801_uploade...
2,Ensemble homme,13 000,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/5313236_uploade...
3,Jean homme,7 000,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/5314908_uploade...
4,Ensemble homme,13 000,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/5314912_uploade...
...,...,...,...,...
375,Jalaba homme,7 000,"Parcelle Assainies, Dakar, Sénégal",https://images.coinafrique.com/5764002_uploade...
376,Abaya vip courte manche avec pantalon,12 000,"Parcelle Assainies, Dakar, Sénégal",https://images.coinafrique.com/5764009_uploade...
377,Abaya xaftan coton long manche avec pantalon,11 000,"Parcelle Assainies, Dakar, Sénégal",https://images.coinafrique.com/5768088_uploade...
378,Djalabe homme,8 500,"Fass, Dakar, Sénégal",https://images.coinafrique.com/5737158_uploade...


## 3. je generalise avec une fonction

In [ ]:

import os

def nettoyer_et_exporter_csv(chemin_csv):
    """
    Nettoie un fichier CSV extrait depuis webscraper et l'exporte automatiquement avec '_clean.csv'.
    
    - Supprime certaines colonnes inutiles
    - Nettoie la colonne 'price' (ignore les valeurs non numériques comme 'Prix sur demande')
    - Extrait les URLs de la colonne 'url_image'
    
    """
    # Charger le fichier CSV
    data = pd.read_csv(chemin_csv)
    
    # Supprimer les colonnes inutiles
    colonnes_supprimer = ["web_scraper_order", "web_scraper_start_url", "container_links"]
    data.drop(columns=colonnes_supprimer, axis=1, inplace=True, errors='ignore')
  

    def nettoyer_price(valeur):
        try:
            # On enlève " CFA" et les espaces
            return valeur.replace(" CFA", "").strip()
        except AttributeError:
            # Si val n'est pas une string (ex: Prix sur demande ou NaN), on renvoie tel quel
            return valeur
        
    data['price'] = data['price'].apply(nettoyer_price)
    
    # je nettoie la colonne 'url_image'
    if 'url_image' in data.columns:
        temp = data['url_image'].str.split('url("', regex=False)
        data['url_image'] = temp.str[1].str.split('")', regex=False).str[0]

    # je nettoie les doublons et les valeurs manquantes
    data.drop_duplicates(inplace=True)
    data.dropna(inplace=True)
    # Construire le nouveau nom de fichier
    base, ext = os.path.splitext(chemin_csv)
    nouveau_fichier = f"{base}_clean.csv"
    
    # Exporter le DataFrame nettoyé
    data.to_csv(nouveau_fichier, index=False)
    
    print(f"Fichier nettoyé enregistré sous : {nouveau_fichier}")
    return nouveau_fichier


## j'execute la fonction 

In [7]:
fichiers = [f"donnée web scrapper/data_url{i}.csv" for i in range(1, 5)]

for f in fichiers: 
    nettoyer_et_exporter_csv(f)


Fichier nettoyé enregistré sous : donnée web scrapper/data_url1_clean.csv
Fichier nettoyé enregistré sous : donnée web scrapper/data_url2_clean.csv
Fichier nettoyé enregistré sous : donnée web scrapper/data_url3_clean.csv
Fichier nettoyé enregistré sous : donnée web scrapper/data_url4_clean.csv


## je stocke les données dans la base de données

In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("BD_coinafrique.db")
cursor = conn.cursor()


In [9]:

fichiers = [1, 2, 3, 4]

# Lire tous les fichiers bruts
dfs_brutes = []
for i in fichiers:
    csv_brut = f"donnée web scrapper/data_url{i}.csv"
    dfs_brutes.append(pd.read_csv(csv_brut))

# Concaténer (pandas gère automatiquement les colonnes manquantes)
data_brutes_final = pd.concat(dfs_brutes, ignore_index=True)

# Lire tous les fichiers clean
dfs_clean = []
for i in fichiers:
    csv_clean = f"donnée web scrapper/data_url{i}_clean.csv"
    dfs_clean.append(pd.read_csv(csv_clean))

# Concaténer
data_clean_final = pd.concat(dfs_clean, ignore_index=True)

# Insérer une seule fois
data_brutes_final.to_sql("data_brutes", conn, if_exists="replace", index=False)
data_clean_final.to_sql("data_clean", conn, if_exists="replace", index=False)

conn.close()
print("Toutes les données ont été insérées dans la base SQLite !")

Toutes les données ont été insérées dans la base SQLite !
